In [141]:
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd

## Datamining to build the graph

In [2]:
data = pd.read_csv('data\wiki-topcats-reduced.txt', sep="	", header=None)
data.columns = ["from", "to"]
list1 = data["from"].values
list2 = data["to"].values

Our intial data are rappresented in a series of edges between nodes.

In [3]:
data.head()

,from,to
0,52,401135
1,52,1069112
2,52,1163551
3,62,12162
4,62,167659


The first thing we want is to obtain a list of nodes: we created two sets (which don't contain duplicates) with the nodes included in the two columns and then the union of the two set in a single final set. The lenght of this set is equal to the number of nodes.

In [4]:
set1=set(list1)
set2=set(list2)
fin_set = set1.union(set2)
print(len(fin_set))

461193


A simple check to know wheter the graph is directed or not is to calculate the lenght of both sets: if they are different we can say that it is actually directed.

In [5]:
print(len(set1))
print(len(set2))

428957
352518


Since the lenghts are different we can say the graph is directed.
We want to store it using Networkx library:

In [6]:
G = nx.DiGraph()
for count, line in enumerate(open("data/wiki-topcats-reduced.txt")):
            grlst=line.replace("\n", "").split(sep="\t")
            G.add_edge(int(grlst[0]),int(grlst[1]))

In [145]:
a= sorted(list(G.nodes()))
len(a)

461193

In [54]:
G.add_nodes_from(fin_set)
for i in range(len(data)):
    G.add_edge(data.iloc[i]['from'], data.iloc[i]['to'])

Now we can get some basic informations: number of nodes, number of edges and density of the graph which is calculate as: <br>
<h1><center>$ D = \frac{\lvert E \rvert} {\lvert V \rvert (\lvert V \rvert - 1)}  \simeq \frac{\lvert E \rvert} {V^2}$</center></h1>

In [7]:
nodes = G.number_of_nodes()
edges = G.number_of_edges()
print('Number of nodes: ' + str(nodes))
print('Number of edges: ' + str(edges))
print('Density: ' + str(edges/(nodes**2)))

Number of nodes: 461193
Number of edges: 2645247
Density: 1.243657566949106e-05


In [9]:
list(G.neighbors(52))

[401135, 1069112, 1163551]

Since the value of density is really close to one, we can say that it's sparse graph.

## Datamining to find categories

For our purpose we need to consider now the categories: our firts filter is a number of elements of at least 3500 for the ones we want to take into account.

In [10]:
data_categories = []
with open('data/wiki-topcats-categories.txt') as f:
    for i, line in enumerate(f):
        tmp = line.split(';')
        if len(tmp[1]) > 3500:
            a=[tmp[0].replace('Category:', '')]
            a.append([int(i) for i in tmp[1].split()])
            data_categories.append(a)

f.close()


We this first filter we obtained 1616 categories.

In [125]:
len(data_categories)

1616

But we also want to remove all the nodes that are not in our reduced graph:

In [14]:
data_cat_fin = []
for i in range(len(data_categories)):
    data_categories[i][1] = fin_set.intersection(data_categories[i][1])
    if len(data_categories[i][1]) > 3500:
        data_cat_fin.append(data_categories[i])
        
        
print(len(data_cat_fin))

29


We'll take into account only 29 categories.

In [147]:
for i in range(len(data_cat_fin)):
    print(len(data_cat_fin[i][1]))

7538
7814
5097
3737
5827
4588
348300
5549
5192
6491
5568
4122
28498
7729
11660
4895
4760
4422
22463
15159
4614
11531
13865
7561
10759
4346
5532
3720
4025


## Block-ranking

We have to choose the input category:

In [17]:
for i in range(len(data_cat_fin)):
    print(len(data_cat_fin[i][1]))
    

7538
7814
5097
3737
5827
4588
348300
5549
5192
6491
5568
4122
28498
7729
11660
4895
4760
4422
22463
15159
4614
11531
13865
7561
10759
4346
5532
3720
4025


## Weight nodes

In [17]:
used_nodes = []

In [128]:
def neighbors (n, list1, list2):
    ''' return the neighbors of n'''
    A = []
    
    if n < 1055832:
        for i in range(int((len(list1)+1)/2)):
            if list1[i] == n:
                A.append(list2[i])
            if list1[i] > n:
                break
                
    else:
        for i in range(len(list1)):
            if list1[len(list1)-i-1] == n:
                A.append(list2[len(list1)-i])
            if list1[len(list1)-i-1] < n:
                break
    
    return A

In [99]:
rank = [4,18,25,12,3,24,19,27,7,13,20,1,0,11,21,26,14,9,23,2,28,10,8,15,22,5,17,16,6]

In [83]:
def weight_nodes (category):
    #creating a vector of tuples which will contain the weight of nodes
    W = []
    for i in category:
        V = [i]
        pred = set(G.predecessors(i))
        a = pred.intersection(category)
        V.append(len(a))
        W.append(V)
    #calculating the weight    
    W = sorted(W,key=lambda x: x[1], reverse = True)  
    return W       

In [108]:
def weight_nodes_est (category, old_cat):
    #creating a vector of tuples which will contain the weight of nodes
    W = []
    for i in category:
        V = [i]
        pred = set(G.predecessors(i))
        a = pred.intersection(category)
        nodes_oc = set([row[0] for row in old_cat])
        b = pred.intersection(nodes_oc)
        c = 0
        for j in range(len(old_cat)):
            for y in b:
                if old_cat[j][0] == y:
                    c = c + old_cat[j][1]
            
            
        V.append(len(a)+c)
        W.append(V)
    #calculating the weight    
        
    W = sorted(W,key=lambda x: x[1], reverse = True)    
    return W 

## Clean categories folowing the rank

In [136]:
used_nodes = data_cat_fin[rank[0]][1]
C = list(used_nodes)

In [140]:
for i in range(len(rank)-1):
    C1 = data_cat_fin[i+1][1].intersection(used_nodes)
    print(len(C1))
    used_nodes = used_nodes.union(C1)
    C.append(list(C1))


1511
170
0
5827
290
5509
0
0
0
0
34
4
0
0
0
0
0
0
0
0
0
0
0
0
8
26
0
0


884755

### Start with the first category

In [109]:
C0 = data_cat_fin[rank[0]][1]

In [110]:
W0 = weight_nodes (C0)

In [111]:
W0

[[89978, 46],
 [81856, 41],
 [82393, 34],
 [83513, 33],
 [81829, 29],
 [83088, 29],
 [81878, 28],
 [78971, 27],
 [81835, 25],
 [81996, 24],
 [81858, 21],
 [1358669, 18],
 [83531, 18],
 [81905, 17],
 [82354, 17],
 [82368, 17],
 [82948, 17],
 [82350, 16],
 [83539, 16],
 [84849, 15],
 [88793, 15],
 [81940, 14],
 [82337, 14],
 [86475, 14],
 [89876, 14],
 [82326, 13],
 [83530, 13],
 [83955, 13],
 [84178, 13],
 [1358640, 12],
 [82318, 12],
 [82587, 12],
 [83542, 12],
 [89877, 12],
 [737917, 11],
 [83081, 11],
 [87589, 11],
 [88801, 11],
 [89700, 11],
 [895635, 10],
 [1358791, 10],
 [81219, 10],
 [81493, 10],
 [84847, 10],
 [86509, 10],
 [86612, 10],
 [88815, 10],
 [895726, 9],
 [76811, 9],
 [1358601, 9],
 [81291, 9],
 [81568, 9],
 [81587, 9],
 [81826, 9],
 [81837, 9],
 [81925, 9],
 [82054, 9],
 [82372, 9],
 [83981, 9],
 [84386, 9],
 [88842, 9],
 [89697, 9],
 [1358646, 8],
 [1358672, 8],
 [1358731, 8],
 [81314, 8],
 [81426, 8],
 [81551, 8],
 [81611, 8],
 [81838, 8],
 [82052, 8],
 [82321, 8],


In [112]:
used_nodes = C0

In [114]:
for i in range(len(rank) - 1):
    new_cat = data_cat_fin[rank[i-1]][1]
    if i == 0:
        old_cat = C0
    else

In [84]:
C0 = data_cat_fin[7][1]
C1 = data_cat_fin[9][1]

In [87]:
C1_new = C1.difference(C0)

In [88]:
len(C1_new)

6478

In [93]:
E = weight_nodes_est (C1_new, a)

In [94]:
E

[[540950, 289],
 [539587, 252],
 [537481, 229],
 [539465, 207],
 [539805, 206],
 [539770, 176],
 [539586, 148],
 [539918, 147],
 [541695, 132],
 [539920, 127],
 [537716, 125],
 [541696, 121],
 [537719, 119],
 [130063, 109],
 [539473, 108],
 [537717, 100],
 [541706, 99],
 [537478, 94],
 [539622, 94],
 [539623, 88],
 [537479, 81],
 [539476, 80],
 [539671, 75],
 [537603, 69],
 [537604, 69],
 [537601, 65],
 [537605, 63],
 [539780, 62],
 [539766, 61],
 [537501, 58],
 [539789, 55],
 [541670, 54],
 [536256, 52],
 [537508, 52],
 [539917, 52],
 [539589, 49],
 [543587, 47],
 [539879, 46],
 [537591, 45],
 [539919, 43],
 [539588, 40],
 [543590, 40],
 [539254, 39],
 [543594, 39],
 [543595, 39],
 [539801, 38],
 [64825, 36],
 [537654, 35],
 [537714, 35],
 [539474, 35],
 [540113, 35],
 [537502, 34],
 [539625, 34],
 [539807, 34],
 [541684, 34],
 [541654, 33],
 [541690, 33],
 [542193, 33],
 [539494, 32],
 [539493, 31],
 [539492, 30],
 [539881, 30],
 [541704, 30],
 [535382, 29],
 [537494, 29],
 [543627, 